# Signature similarity across cell lines

### *Ted Natoli*

The goal of this exercise is to perform a simple analysis of cross-cell line signature similarity for a small collection of compounds. Along the way we will demonstrate some basic functionality in the [cmapR](https://github.com/workshop/data/cmapR) package.

## Signature selection

To keep the dataset size manageable, we selected signatures of 17 compounds that each gave a highly reproducible signature in each of 15 cell lines. So each of the 255 selected signatures under consideration was highly reproducible. Signatures were chosen to have between 2-10 replicates and where the dose was between 1 and 20 uM. In some cases, there was more than one possible signature for each compound/cell line combination. In those cases, we chose the one that was most reproducible.

## Setup

In [ ]:
library(cmapR)
library(pheatmap)
library(RColorBrewer)
library(repr)

In [ ]:
# paths to relevant data files
ds_path <- "/srv/data/workshop/data/Module4/cross_cell_modz_n255x978.gctx"
meta_path <- "/srv/data/workshop/data/Module4/cross_cell_meta.txt"


### Read inputs

In [ ]:
# read the dataset and view contents
(ds <- cmapR::parse.gctx(ds_path))

In [ ]:
# read metadata
meta <- data.table::fread(meta_path)

Apply annotations to the GCT object using the `annotate.gct` method from `cmapR`.

In [ ]:
# annotate the dataset
(ds <- cmapR::annotate.gct(ds, meta, dim="col", keyfield="sig_id"))

We can now see that the dataset has column annotations for all the contained signatures. 

## Compute pairwise correlations

To do this, we'll compute all pairwise correlations between signatures and use that matrix to construct a new GCT object.

In [ ]:
corr_mat <- cor(ds@mat, method="spearman")

In [ ]:
# build a new GCT object from the correlation matrix
# we can take advantage of the fact that the rows/columns of
# corr_mat are in the same order as the original columns of ds
(corr_ds <- new("GCT", mat=corr_mat, cdesc=ds@cdesc, rdesc=ds@cdesc))

## Clustering

Let's try clustering this correlation matrix to see whether the clusters are driven by cell line.

In [ ]:
# first set aside an annotation data.frame to indicate cell line
plot_df <- data.frame(cell_id = corr_ds@cdesc$cell_id)
rownames(plot_df) <- corr_ds@cid

In [ ]:
# plot the clustered heatmap using pheatmap package and overlaying
# the cell line designation as a color bar on the margins
options(repr.plot.width=10, repr.plot.height=7)
pheatmap::pheatmap(corr_ds@mat, annotation_row=plot_df, annotation_col=plot_df,
                  show_rownames=F, show_colnames=F)

In general, we don't see strong clustering by cell line. That is, we don't see contiguous blocks of the same color along the margins. This suggests that perhaps the compounds themselves are what's driving the clustering.

## Quantifying frequency of high correlation

Let's look at the distribution of correlation values, ignoring the trivial self-correlations along the diagonal.

In [ ]:
options(repr.plot.width=5, repr.plot.height=4)
hist(corr_ds@mat[upper.tri(corr_ds@mat, diag=F)],
    xlab="correlation",
    breaks=30,
    col="dodgerblue",
    border="white",
    main="All Pairwise Correlations")

What are the quantiles of these correlation values?

In [ ]:
quantile(corr_ds@mat[upper.tri(corr_ds@mat, diag=F)],
        probs=seq(0, 1, 0.05))

So it seems like ~0.37 is the 95th quantile of correlation. 

### By compound

Let's try to quantify how often a given compound correlates with itself above a this threshold. To do this, we'll melt our `corr_ds` object into a data.frame for easier aggregation.

In [ ]:
melted_corr <- cmapR::melt.gct(corr_ds, remove_symmetries = T)

Let's take a look at this melted object.

In [ ]:
str(melted_corr)

Note the '.x' and '.y' suffixes on the column names to indicate whether the corresponding field originally came from the row or column of `corr_ds`, respectively. 

Let's aggregate the `melted_corr` to ask how often the same compound correlates with itself above 0.37.

In [ ]:
melted_corr_agg <- melted_corr[
    pert_iname.x == pert_iname.y &          # consider cases where it's the same compound
    cell_id.x != cell_id.y,                 # but not the same cell line
    .(
        ncomp = .N,                         # how many comparisons for this compound
        ncomp_hi_corr = sum(value >= 0.37), # how many comparisons are above 0.37
        median_corr = median(value)         # median cross cell correlation 
    ), .(pert_iname.x)]                     # aggregate by pert_iname

In [ ]:
# add a column for the fraction of high-correlation comparisons
# and rename pert_iname.x to pert_iname
melted_corr_agg[, frac_comp_hi_corr := ncomp_hi_corr / ncomp, .(pert_iname.x)]
data.table::setnames(melted_corr_agg, "pert_iname.x", "pert_iname")

In [ ]:
str(melted_corr_agg)

Let's make a barplot of the high correlation frequencies.

In [ ]:
options(repr.plot.width=5.5, repr.plot.height=4.5)
par(mar=c(10, 4, 2, 2)+0.1)
with(melted_corr_agg[order(frac_comp_hi_corr, decreasing=T)],
    barplot(frac_comp_hi_corr, names.arg=pert_iname, las=3,
           ylab="Frequency of High Correlation"))

Perhaps not surprisingly, the HDAC inhibitors most frequently correlate across cell lines. The SRC, HER, JAK inhibitors that correlate least frequently may also be interesting. We know going in that the selected signatures were all individually reproducible, so the fact that they do not correlate strongly across cell lines may suggest that they are having some cell-specific effects.

### By cell line

Let's flip things around and ask for every cell line pair, what fraction of compounds correlate strongly across the two lines. We'll do the aggregation a little differently here, and actually will consider the symmetric values from `corr_ds` for convenience. We'll also lower the threshold slightly so that the results are not driven by only a small number of comparisons.

In [ ]:
# make a new melted data.frame keeping the symmetries
melted_corr_symm <- melt.gct(corr_ds)

In [ ]:
melted_corr_agg_cell <- melted_corr_symm[
    pert_iname.x != pert_iname.y,           # consider cases where it's NOT same compound
    .(
        ncomp = .N,                         # how many comparisons for this compound
        ncomp_hi_corr = sum(value >= 0.20), # how many comparisons are above 0.37
        median_corr = median(value)         # median cross cell correlation 
    ), .(cell_id.x, cell_id.y)]    
# add a column for the fraction of high-correlation comparisons
# divide by 2 to account for the symmetries
melted_corr_agg_cell[, frac_comp_hi_corr := (ncomp_hi_corr / ncomp) / 2,
                .(cell_id.x, cell_id.y)]

In [ ]:
str(melted_corr_agg_cell)

Let's look at this as a heatmap.

In [ ]:
# first convert to a matrix
tmp <- data.table::dcast(melted_corr_agg_cell, cell_id.x ~ cell_id.y,
                        value.var="frac_comp_hi_corr")
cross_cell_hicorr_freq_mat <- as.matrix(tmp[, -1])
rownames(cross_cell_hicorr_freq_mat) <- tmp[[1]]

In [ ]:
str(cross_cell_hicorr_freq_mat)

In [ ]:
pheatmap::pheatmap(cross_cell_hicorr_freq_mat,
                   color = colorRampPalette(rev(RColorBrewer::brewer.pal(n = 7, name="RdYlBu")))(25),
                   breaks=seq(0, 0.25, 0.01),
                   display_numbers=T)

Overall the frequencies seem pretty low, but but there are perhaps some interesting patterns that emerge. Let's just look at the dendrogram to see the clustering a bit more clearly.

In [ ]:
plot(hclust(as.dist(1-cross_cell_hicorr_freq_mat)), xlab="")

In general the two large clusters seem to make sense. The one on the right contains majority non-cancer lines (other than HELA). The two breast cancer lines (MCF7 + MDAMB231) cluster together but apart from MCF10A, an immortalized breast line.